# Felony Modeling

Code for Charlottesville is collaborating with the LAJC to examine Virginia criminal court data and advocate for expungement of certain criminal records. As part of this project, they are interested in whether and how racial bias may play a role in case outcomes. One question of interest is what factors are predictive of whether someone is charged with a felony or a misdemeanor for a particular crime.

To investigate this question, we implement logistic regression and random forest classification models using felony/misdemeanor as the target predictor variable. We use race along with other predictor variables to see how much impact race has on charge type. We are particularly interested in marijuana charges, since the recent legalization of marijuana in Virginia has made prior marijuana charges a particular priority for expungement.

### Summary

Using these techniques, we found that race did not make much difference in the rates at which people were charged with felonies or misdemeanors for charges of marijuana possession with intent to distribute. However, race was more predictive than gender as a predictive demographic attribute for misdemeanor/felony charges.

### Read in and Pre-process Data

We begin by looking at the district criminal data from 2019. This data contains just under 2 million records, of which about 650,000 are classified as either a misdemeanor or felony.

In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql import functions as F

import json
import pandas as pd
import matplotlib.pyplot as plt

spark = SparkSession.builder.getOrCreate()

In [2]:
circuit = spark.read\
                .format("csv")\
                .option("header", "true")\
                .load("circuit/circuit_criminal_20**_anon_*.csv")

In [3]:
circuit.take(2)

[Row(HearingDate='2020-02-27', HearingResult='Nolle Prosequi', HearingJury=None, HearingPlea=None, HearingType='Plea', HearingRoom=None, fips='99', Filed='2018-03-14', Commencedby='Direct Indictment', Locality='COMMONWEALTH OF VA', Sex='Male', Race='Black', Address='LEXINGTON PARK, MD 20653', Charge='FIREARM: USE IN COMMIS OF FEL', CodeSection='18.2-53.1', ChargeType='Felony', Class='U', OffenseDate='2018-01-06', ArrestDate='2019-11-23', DispositionCode='Nolle Prosequi', DispositionDate='2020-02-27', ConcludedBy='Nolle Prosequi', AmendedCharge=None, AmendedCodeSection=None, AmendedChargeType=None, JailPenitentiary=None, ConcurrentConsecutive=None, LifeDeath=None, SentenceTime=None, SentenceSuspended=None, OperatorLicenseSuspensionTime=None, FineAmount=None, Costs=None, FinesCostPaid=None, ProgramType=None, ProbationType=None, ProbationTime=None, ProbationStarts=None, CourtDMVSurrender=None, DriverImprovementClinic=None, DrivingRestrictions=None, RestrictionEffectiveDate=None, Restricti

In [4]:
circuit.count()

3634322

In [5]:
#most fields are strings, but wanted to create schema so that a few columns are numeric
fields = []
for f in json.loads(circuit.schema.json())["fields"]:
    if f["name"] in ['SentenceTime', 'ProbationTime', 'Costs', 'FineAmount', 'FineCostsPaid']:
        fields.append(StructField(f["name"], DoubleType(), True))
    else:
        fields.append(StructField.fromJson(f))

schema = StructType(fields)

In [6]:
circuit = spark.read.schema(schema)\
                .format("csv")\
                .option("header", "true")\
                .load("circuit/circuit_criminal_20**_anon_*.csv")

In [7]:
#cleaning/standardizing race names

circuit = circuit.withColumn('RaceClean', F.regexp_replace('Race', '\(Non-Hispanic\)', ''))
circuit = circuit.withColumn('RaceClean', F.regexp_replace('RaceClean', ' Caucasian', ''))
circuit = circuit.withColumn('RaceClean', F.regexp_replace('RaceClean', 'Asian Or', 'Asian or'))
circuit = circuit.withColumn('RaceClean', F.regexp_replace('RaceClean', ' \(Includes Not Applicable, Unknown\)', ''))
circuit = circuit.withColumn('RaceClean', F.regexp_replace('RaceClean', '\(Includes Not Applicable, Unknown\)', ''))
circuit = circuit.withColumn('RaceClean', F.regexp_replace('RaceClean', 'Other', 'Unknown'))

In [8]:
data = circuit.filter(circuit.ChargeType.isin(['Misdemeanor', 'Felony']))

In [9]:
data = data.withColumn('Felony', F.when(data.ChargeType == 'Felony', 1).otherwise(0))

In [10]:
data.count()

3438815

In [11]:
circuit.count()

3634322

### Logistic Regression Model

To test whether racial bias influences the outcomes, we create a logistic regression model to predict case type (misdemeanor/felony) using code section (code representing crime charged), gender, race, and plea type (innocent/guilty/Nolo Contendere/etc).

We chose to use a lasso model because the lasso penalty encourages model coefficients to be equal to zero when they are not contributing significantly to the model. Thus if the coefficients for race are non-zero, we have some evidence that race is useful in predicting whether a crime is a misdemeanor or a felony.

In [12]:
from pyspark.ml import Pipeline  
from pyspark.ml.feature import *
from pyspark.ml.classification import LogisticRegression

In [13]:
#all predictor variables are categorical and need to be one-hot encoded before modeling

gendInd = StringIndexer(inputCol="Sex", outputCol="GendInd", handleInvalid = "skip")
gend = OneHotEncoder(inputCol="GendInd", outputCol="GenderOH")

raceInd = StringIndexer(inputCol="RaceClean", outputCol="RaceInd", handleInvalid = "skip")
race = OneHotEncoder(inputCol="RaceInd", outputCol="RaceOH")

chargeInd = StringIndexer(inputCol="CodeSection", outputCol="ChargeInd", handleInvalid = "skip")
charge = OneHotEncoder(inputCol="ChargeInd", outputCol="ChargeCodeOH")

fipsInd = StringIndexer(inputCol="fips", outputCol="FipsInd", handleInvalid = "skip")
fips = OneHotEncoder(inputCol="FipsInd", outputCol="FipsOH")

#gather encoded predictors into features vector
va = VectorAssembler(inputCols=["RaceOH", "ChargeCodeOH", "FipsOH", "GenderOH"], outputCol="features", 
                     handleInvalid = "skip")

logm = LogisticRegression(labelCol = 'Felony', elasticNetParam = 1) #lasso = 1, ridge = 0

In [14]:
pipeline = Pipeline(stages=[gendInd, gend, raceInd, race, chargeInd, charge, fipsInd, fips, va, logm])

Normally, we would split the data into training and testing sets, but since we're primarily interested in model interpretation rather than prediction, we go ahead and train on the full dataset here.

In [15]:
seed = 34
weights = [0.9, 0.1]
train, test = data.randomSplit(weights, seed)

In [16]:
model = pipeline.fit(data)
pred = model.transform(data)

In [42]:
trainingSummary = model.stages[-1].summary

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
# trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

areaUnderROC: 0.9595478928357579


In [43]:
trainingSummary.accuracy

0.9181100114338171

In [44]:
trainingSummary.weightedPrecision

0.9170064803060798

In [45]:
trainingSummary.weightedRecall
# https://spark.apache.org/docs/2.4.5/api/python/pyspark.ml.html?highlight=coefficients#pyspark.ml.classification.LogisticRegressionModel.coefficients

0.9181100114338171

The AUC tells us that this model is quite successful in predicting whether a charge is a felony or misdemeanor from charge, plea, race, and gender.

In [20]:
#figure out which coefficients map to which characteristics

# https://stackoverflow.com/questions/39022052/relating-column-names-to-model-parameters-in-pyspark-ml

# numeric_metadata = pred.select("features").schema[0].metadata.get('ml_attr').get('attrs').get('numeric')
binary_metadata = pred.select("features").schema[0].metadata.get('ml_attr').get('attrs').get('binary')

# merge_list = numeric_metadata + binary_metadata 
binary_metadata[:7]

[{'idx': 0, 'name': 'RaceOH_White '},
 {'idx': 1, 'name': 'RaceOH_Black '},
 {'idx': 2, 'name': 'RaceOH_White'},
 {'idx': 3, 'name': 'RaceOH_Black'},
 {'idx': 4, 'name': 'RaceOH_Hispanic'},
 {'idx': 5, 'name': 'RaceOH_Unknown'},
 {'idx': 6, 'name': 'RaceOH_Asian or Pacific Islander'}]

In [21]:
print(model.stages[-1].coefficients[0])
print(model.stages[-1].coefficients[1])
print(model.stages[-1].coefficients[2])
print(model.stages[-1].coefficients[3])
print(model.stages[-1].coefficients[4])
print(model.stages[-1].coefficients[5])

0.35009240917411016
0.5674705866767243
0.4614170738434217
0.4938872734327986
0.20520803563337128
0.3152720190694764


None of the race coefficients are equal to 0, so this is evidence that race does play at least some role in whether someone is charged with a misdemeanor or felony.

In [22]:
pred.filter(pred.RaceClean == 'White').filter(pred.Sex == 'Male')\
        .filter(pred.CodeSection == '18.2-248.1')\
        .filter(pred.fips == '3')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='White', Sex='Male', probability=DenseVector([0.0781, 0.9219]), prediction=1.0)]

In [23]:
pred.filter(pred.RaceClean == 'Black').filter(pred.Sex == 'Male')\
        .filter(pred.CodeSection == '18.2-248.1')\
        .filter(pred.fips == '3')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='Black', Sex='Male', probability=DenseVector([0.0758, 0.9242]), prediction=1.0)]

In [24]:
pred.filter(pred.RaceClean == 'White').filter(pred.Sex == 'Female')\
        .filter(pred.CodeSection == '18.2-248.1')\
        .filter(pred.fips == '3')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='White', Sex='Female', probability=DenseVector([0.0844, 0.9156]), prediction=1.0)]

In [25]:
pred.filter(pred.RaceClean == 'Black').filter(pred.Sex == 'Female')\
        .filter(pred.CodeSection == '18.2-248.1')\
        .filter(pred.fips == '3')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='Black', Sex='Female', probability=DenseVector([0.0819, 0.9181]), prediction=1.0)]

This model predicts roughly the same chance of getting charged with a felony for Black and white men. Other races have a small sample size, so we did not consider them here.

In [26]:
pred.filter(pred.RaceClean == 'White').filter(pred.Sex == 'Male')\
        .filter(pred.CodeSection == '18.2-250')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='White', Sex='Male', probability=DenseVector([0.0595, 0.9405]), prediction=1.0)]

In [27]:
pred.filter(pred.RaceClean == 'Black').filter(pred.Sex == 'Male')\
        .filter(pred.CodeSection == '18.2-250')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='Black', Sex='Male', probability=DenseVector([0.0587, 0.9413]), prediction=1.0)]

## Model with Just Marijuana Charges

Next, we create a similar model using just the data for marijuana possession with intent to distribute (code section 18.2-248.1). This charge can be either a misdemeanor or a felony, while marijuana possession (18.2-250.1) is always a misdemeanor. This subset of the data contains 4445 records.

In [28]:
data_mj = data.filter(data.CodeSection.isin(['18.2-248.1'])) 
# this can be felony or misdemeanor (marijuana poss w/ intent to distribute)
# 18.2-250.1 is always a misdemeanor (possession)

In [29]:
data_mj.count()

43104

In [30]:
seed = 34
weights = [0.9, 0.1]
train, test = data_mj.randomSplit(weights, seed)

In [31]:
#remove charge code as a predictor since just one charge included now
#one-hot encoding and logistic modeling steps can stay the same

va = VectorAssembler(inputCols=["RaceOH", "GenderOH"], outputCol="features", 
                     handleInvalid = "skip")

pipeline_mj = Pipeline(stages=[gendInd, gend, raceInd, race, va, logm])

In [32]:
model_mj = pipeline_mj.fit(train)

In [33]:
pred_mj = model_mj.transform(test)

In [34]:
trainingSummary = model_mj.stages[-1].summary

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
# trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

areaUnderROC: 0.5287797874065792


In [41]:
trainingSummary.accuracy

0.882808252867017

In [39]:
trainingSummary.weightedRecall

0.882808252867017

In [40]:
trainingSummary.weightedPrecision

0.7793504113301151

In [35]:
model_mj.stages[-1].coefficients

DenseVector([-12.9984, -13.2006, -12.9536, -13.1168, -13.3737, -13.104, -12.734, 10357.643, 0.0842])

As in the previous model, none of the coefficients for race are equal to zero, although with so few variables in the model, this isn't as meaningful as if race was a significant predictor in the presence of many other predictors.

In [32]:
pred_mj.filter(pred.RaceClean == 'White').filter(pred.Sex == 'Male')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='White', Sex='Male', probability=DenseVector([0.1175, 0.8825]), prediction=1.0)]

In [33]:
pred_mj.filter(pred.RaceClean == 'Black').filter(pred.Sex == 'Male')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='Black', Sex='Male', probability=DenseVector([0.1018, 0.8982]), prediction=1.0)]

In [34]:
pred_mj.filter(pred.RaceClean == 'Hispanic').filter(pred.Sex == 'Male')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='Hispanic', Sex='Male', probability=DenseVector([0.0822, 0.9178]), prediction=1.0)]

In [35]:
pred_mj.filter(pred.RaceClean == 'White').filter(pred.Sex == 'Female')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='White', Sex='Female', probability=DenseVector([0.1252, 0.8748]), prediction=1.0)]

In [36]:
pred_mj.filter(pred.RaceClean == 'Black').filter(pred.Sex == 'Female')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='Black', Sex='Female', probability=DenseVector([0.1086, 0.8914]), prediction=1.0)]

In [37]:
pred_mj.filter(pred.RaceClean == 'Hispanic').filter(pred.Sex == 'Female')\
        .select('Race', 'Sex', 'probability', 'prediction').take(1)

[Row(Race='Hispanic', Sex='Female', probability=DenseVector([0.0878, 0.9122]), prediction=1.0)]

Since this model only uses race and gender as predictors, the prediction will be the same for every white male and every black male (and every other combination of race and gender).

In this very basic model, a Black man is predicted to have an 85% chance of being charged with a felony for possession of marijuana, while a white man is predicted to have an 82% chance. 

Below, we look at the percentages in the data to sanity-check our model.

In [38]:
fel = data.groupBy('RaceClean').agg(F.count(data.RaceClean).alias('count'), 
                                    F.sum(data.Felony).alias('n_felonies'))

fel.withColumn('percent', fel['n_felonies']/fel['count']).show()

+--------------------+-------+----------+------------------+
|           RaceClean|  count|n_felonies|           percent|
+--------------------+-------+----------+------------------+
|              Black |1310591|    983180|0.7501806436943333|
|                null|      0|      5509|              null|
|             Unknown|  23864|     15665|0.6564280925243043|
|     American Indian|   1416|       863|0.6094632768361582|
|American Indian O...|    122|        69|0.5655737704918032|
|               White| 252156|    191621|0.7599303605704405|
|               Black| 162018|    120484|0.7436457677542001|
|            Hispanic|  28866|     18840| 0.652670962377884|
|              White |1636003|   1196647|0.7314454802344494|
|Asian or Pacific ...|  12390|      8203|0.6620661824051655|
+--------------------+-------+----------+------------------+



In [39]:
fel = data.filter(data.CodeSection == '18.2-248.1').groupBy('RaceClean')\
            .agg(F.count(data.RaceClean).alias('count'), 
                 F.sum(data.Felony).alias('n_felonies'))

fel.withColumn('percent', fel['n_felonies']/fel['count']).show()

+--------------------+-----+----------+------------------+
|           RaceClean|count|n_felonies|           percent|
+--------------------+-----+----------+------------------+
|              Black |18297|     15963|0.8724381046073126|
|                null|    0|        45|              null|
|             Unknown|  409|       347|0.8484107579462102|
|     American Indian|   18|        18|               1.0|
|American Indian O...|    2|         2|               1.0|
|               White| 2149|      1894|0.8813401582131224|
|               Black| 2426|      2178|0.8977741137675186|
|            Hispanic|  291|       267|0.9175257731958762|
|              White |19158|     17074|0.8912203779100115|
|Asian or Pacific ...|  302|       267|0.8841059602649006|
+--------------------+-----+----------+------------------+



For those charged with marijuana possession, 80% of whites and 77% of Blacks were charged with a felony, both of which are slightly lower than predicted by the model (for both genders).

Other races have very small sample sizes, so we can't really draw many conclusions from that.

## Random Forest Feature Importance

Next, we implement a random forest model to quantify the feature importance further.

In [11]:
from pyspark.ml.classification import RandomForestClassifier

In [12]:
rf = RandomForestClassifier(labelCol="Felony", featuresCol="features", numTrees=100)

In [13]:
va = VectorAssembler(inputCols=["RaceOH", "ChargeCodeOH", "FipsOH", "GenderOH"], outputCol="features", 
                     handleInvalid = "skip")

pipeline_rf = Pipeline(stages=[gendInd, gend, raceInd, race, chargeInd, charge, fipsInd, fips, va, rf])

In [14]:
model_rf = pipeline_rf.fit(data)
pred_rf = model_rf.transform(data)

In [15]:
# model_rf.stages[-1].featureImportances

In [16]:
meta_rf = pred_rf.select("features").schema[0].metadata.get('ml_attr').get('attrs').get('binary')

# merge_list = numeric_metadata + binary_metadata 
feature_names = [field['name'] for field in meta_rf]

In [17]:
list(zip(model_rf.stages[-1].featureImportances.toArray(), feature_names))[:15]

[(0.0011234862147014152, 'RaceOH_White '),
 (0.0, 'RaceOH_Black '),
 (0.0, 'RaceOH_White'),
 (0.0, 'RaceOH_Black'),
 (0.0, 'RaceOH_Hispanic'),
 (0.0, 'RaceOH_Unknown'),
 (0.0, 'RaceOH_Asian or Pacific Islander'),
 (0.0, 'RaceOH_American Indian'),
 (0.008468227666377207, 'ChargeCodeOH_19.2-306'),
 (0.033416539092740935, 'ChargeCodeOH_18.2-250'),
 (0.02684488376714018, 'ChargeCodeOH_18.2-248'),
 (0.03753710572226893, 'ChargeCodeOH_18.2-95'),
 (0.018273086239853614, 'ChargeCodeOH_18.2-172'),
 (0.017756437683214703, 'ChargeCodeOH_18.2-456'),
 (0.008202294997612417, 'ChargeCodeOH_19.2-128')]

In [18]:
list(zip(model_rf.stages[-1].featureImportances.toArray(), feature_names))[-1]

(0.0, 'GenderOH_Male')

In [19]:
feat_imp = list(zip(model_rf.stages[-1].featureImportances.toArray(), feature_names))

feat_imp.sort(key = lambda x: -x[0])

In [20]:
feat_imp[72]

(0.0033212421720506865, 'ChargeCodeOH_18.2-258')

In [23]:
import pandas as pd

feat = pd.DataFrame(feat_imp)
feat.columns = ['importance', 'feature']

# feat.to_csv('feature_importance.csv')

The only races with a non-zero feature importance for predicting whether a crime will be a felony or not are Black, Hispanic and unknown. Several charges have much higher feature importance, and gender also has non-zero feature importance.

This matches expectation, since some charges will obviously have much higher rates of felonies than other charges, and some charges will always be either a misdemeanor and other will always be a felony.

Next, we again look at just marijuana charges for feature importance.

In [ ]:
va = VectorAssembler(inputCols=["RaceOH", "GenderOH"], outputCol="features", 
                     handleInvalid = "skip")

pipeline_mj = Pipeline(stages=[gendInd, gend, raceInd, race, va, rf])

In [ ]:
model_rf = pipeline_mj.fit(data_mj)
pred_rf = model_rf.transform(data_mj)

In [ ]:
#get feature names

meta_rf = pred_rf.select("features").schema[0].metadata.get('ml_attr').get('attrs').get('binary')

# merge_list = numeric_metadata + binary_metadata 
feature_names = [field['name'] for field in meta_rf]

In [ ]:
#combine names with importances
list(zip(feature_names, model_rf.stages[-1].featureImportances))

From this, we see that race is a more important factor than gender in determining whether marijuana possession with intent to distribute is a felony or a misdemeanor.